In [1]:
import sys
sys.path.append(r'C:\Users\damod\OneDrive\Área de Trabalho\Projetos Python\dataprocessing')

Importação das bibliotecas

In [2]:
import dataprocessing.datacolect as dc
import re
import pandas as pd
import numpy as np

# 1. Sobre coleta dos dados na Wikipedia

# 2. Rotina de ***scraping***

A lista de jogos do ***Xbox One*** na ***Wikipedia*** é dividida em jogos de ***A-L e M-Z***. Portanto, as rotinas serão executadas em paralelo neste ***notebook***.

In [3]:
urls = [
    r'https://en.wikipedia.org/wiki/List_of_Xbox_One_games_(A%E2%80%93L)',
    r'https://en.wikipedia.org/wiki/List_of_Xbox_One_games_(M%E2%80%93Z)'
]

In [4]:
soups = dict()

for index, url in enumerate(urls):
    soup_class = dc.Soup(url)
    soups[index] = soup_class.get_soup()

In [5]:
scrape = dc.Scrape()

In [6]:
games_tables = dict()

for key, soup in soups.items():
    games_table = scrape.find_elements(
    tag='tbody', soup=soup,
    )
    games_table = games_table[1]
    games_tables[key] = games_table

In [7]:
games_datas = dict()

for key, games_table in games_tables.items():
    games_data = scrape.find_elements(
    tag='tr', soup=games_table,
    )
    games_data = games_data[2:]
    games_datas[key] = games_data

In [13]:
def scraping_games(games: list, init_info: int=2, last_info: int=4):
    """
    Realiza o scraping das informações dos jogos presentes na tabela da lista da Wikipedia.
    """
    scraped_game = dict()
    scraped_games = list()

    for game in games:
        try:
            name_soup = scrape.find_element(tag='th', soup=game)

            if name_soup == None:
                name_soup = scrape.find_element(tag='td', soup=game)
            name = re.sub(r'\n', '', name_soup.get_text())
            scraped_game['name'] = name
            another_infos = scrape.find_elements(tag='td', soup=game)
            games_info = list()

            for another_info in another_infos[init_info:last_info]:
                info = re.sub(r'\n', '', another_info.get_text())
                games_info.append(info)
            scraped_game['infos'] = games_info
            scraped_games.append(scraped_game.copy())
        except AttributeError as e:
            print(f"Attribute Error: {e}")
    return scraped_games

In [14]:
wiki_games = dict()

for key, games in games_datas.items():
   wiki_games[key] = scraping_games(games)

In [15]:
games_from_a_to_l = pd.DataFrame(wiki_games[0])
games_from_m_to_z = pd.DataFrame(wiki_games[1])

In [16]:
wiki_xbox_one = pd.concat([games_from_a_to_l, games_from_m_to_z])

In [17]:
wiki_xbox_one['infos']

0                                 [Beatshapers, Digerati]
1       [One Guy Games, The Quantum Astrophysicists Gu...
2                            [EastAsiaSoft, EastAsiaSoft]
3                                  [Eko (Interlude), Eko]
4                      [Skobbejak Games, Skobbejak Games]
                              ...                        
1387      [High Voltage Software, GameMill Entertainment]
1388                            [Dotemu, Lucasfilm Games]
1389           [Frontier Developments, Xbox Game Studios]
1390                    [Asobo Studio, Xbox Game Studios]
1391                    [Zoë Mode, Majesco Entertainment]
Name: infos, Length: 2930, dtype: object

In [18]:
wiki_xbox_one['developer'] = wiki_xbox_one['infos'].apply(lambda x: x[0])
wiki_xbox_one['publisher'] = wiki_xbox_one['infos'].apply(lambda x: x[1])
wiki_xbox_one = wiki_xbox_one.drop(columns='infos')

In [20]:
wiki_xbox_one.to_csv(r'C:\Users\damod\OneDrive\Área de Trabalho\Projetos Python\gamepass\data\xbox_one_wikipedia.csv', index=False)